In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
df_solar = pd.read_csv("spg.csv")

In [ ]:
# Separate features and target
X = df_solar.drop(columns=["generated_power_kw"])
y = df_solar["generated_power_kw"]

In [ ]:
# Scale features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

In [ ]:
# Scale target
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [ ]:
# Reshape for LSTM (samples, timesteps, features)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [ ]:
# Build LSTM-CNN Model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_split=0.2, callbacks=[early_stop], verbose=1)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.7719 - val_loss: 0.2834
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2690 - val_loss: 0.2367
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2499 - val_loss: 0.2312
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2254 - val_loss: 0.2286
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2354 - val_loss: 0.2230
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2133 - val_loss: 0.2187
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2098 - val_loss: 0.2157
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2056 - val_loss: 0.2183
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2162 - val_loss: 0.2185
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1979 - val_loss: 0.2167
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1907 - val_loss: 0.2184
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1891 - val_l

In [ ]:
# Predict
y_pred_scaled = model.predict(X_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


In [ ]:
# Inverse transform predictions
y_test_rescaled = scaler_y.inverse_transform(y_test)
y_pred_rescaled = scaler_y.inverse_transform(y_pred_scaled)

In [ ]:
# Evaluate
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
mae=mean_absolute_error(y_test_rescaled, y_pred_rescaled)
rmse=np.sqrt(mse)

In [ ]:
print("🌟 CNN-LSTM Model Evaluation:")
print("Mean Squared Error:", mse)
print("R² Score:", r2)
print("Mean Absolute Error:", mae)
print("Root Mean Square Error:", rmse)

🌟 CNN-LSTM Model Evaluation:
Mean Squared Error: 198856.0336826781
R² Score: 0.782305504727185
Mean Absolute Error: 275.36582297338356
Root Mean Square Error: 445.93276811945327


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

# Inverse transform from scaled to original units
y_test_rescaled = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_rescaled = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))

# Calculate metrics in original units
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled) * 100  # in %

# Display results
print(f"✅ Mean Squared Error (MSE): {mse:.4f}")
print(f"✅ Mean Absolute Error (MAE): {mae:.4f}")
print(f"✅ Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"✅ R-squared (R²): {r2:.4f}")
print(f"✅ Mean Absolute Percentage Error (MAPE): {mape:.4f}%")

✅ Mean Squared Error (MSE): 198856.0337
✅ Mean Absolute Error (MAE): 275.3658
✅ Root Mean Squared Error (RMSE): 445.9328
✅ R-squared (R²): 0.7823
✅ Mean Absolute Percentage Error (MAPE): 15997.2878%


In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled, label='Actual')
plt.plot(y_pred_rescaled, label='Predicted')
plt.title('Actual vs Predicted Power Output')
plt.xlabel('Samples')
plt.ylabel('Generated Power (kW)')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()